# Interventions!

In [ ]:
 from julia import Julia
jl = Julia(sysimage = "/home/callum/ASF/Fitting/sys_model.so") #loading sys image

In [ ]:
import matplotlib.pyplot as plt

import os
import tempfile
import numpy as np
import scipy as sp

from scipy import stats
import random as rd
from brokenaxes import brokenaxes

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 10
})

In [ ]:
plt.style.use('seaborn-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
def read_inputs(path,number):
    
    models = [1,2,3,4]
    
    if number not in models:
        raise Exception("Model number must be 1, 2, 3 or 4") 
        
    weights = np.genfromtxt(path + "/L{}_weights.csv".format(number), delimiter=',')
    params = np.genfromtxt(path + "/L{}_params.csv".format(number), delimiter=',')
    
    nparams = params.shape[1]
    
    p1_dis = stats.gaussian_kde(params[:,0],weights = weights)
    p2_dis = stats.gaussian_kde(params[:,1],weights = weights)
    
    if nparams == 2:
        return p1_dis, p2_dis
    elif nparams == 3:
        p3_dis = stats.gaussian_kde(params[:,2],weights = weights)
        return p1_dis, p2_dis, p3_dis
    

In [ ]:
def distance_from_mean(store):
    
    mean_ep = 1.5
    std_ep = 0.604

    mean_pd = 75
    std_pd = 6.08

    mean_mt = 180
    std_mt = 36.475

    
    store_t = (store -[mean_ep, mean_pd, mean_mt]) / ([std_ep, std_pd, std_mt])
    
    d = np.linalg.norm(store_t,axis=1)
    
    mean = np.mean(d)
    c_95 = np.percentile(d,[2.5, 97.5])
    
    print('Mean distance = ', mean)
    print('95% confidence interval =', c_95)
    
    return d, mean, c_95

In [ ]:
def run_simulations_inter(path,number,n_sims, value, network = "none", median = False):
    
    posteriors = read_inputs(path,number)
    
    n_params = len(posteriors) #two or three fitted parameters!
    
    dd = 0
    n_samples = 100000
    
    dp1 = posteriors[0].resample(n_samples)[0]
    dp2 = posteriors[1].resample(n_samples)[0]

    rd.shuffle(dp1)
    rd.shuffle(dp2)
    
    if n_params == 3:
        dp3 = posteriors[2].resample(n_samples)[0]
        rd.shuffle(dp3)
        
    if median:
        if n_params == 2:
            p = (np.median(dp1), np.median(dp2), value,number)
            
        elif n_params ==3:
             p = (np.median(dp1), np.median(dp2), np.median(dp3), value,number)
            
    for i in range(n_sims):
        
        if not median:
            if n_params == 2:
                p = (dp1[i], dp2[i], value,number)
            elif n_params ==3:
                p = (dp1[i], dp2[i], dp3[i], value,number)

        
        if n_params == 2:
            out = model_int(p)
          
        else:
            out = model_int(p, network)
        
        dd += out
  
    return dd


In [ ]:
def simulate_intervention(mn,nsims,path, cs):
   
    dummy  = np.zeros(len(cs)) 
    for i, j in enumerate(cs):

        wa = (100-j)/100
        
        val = run_simulations_inter(path,mn,nsims, wa)
        red = round(100*val/n_sims,3)
        dummy[i] = red
        
    return dummy

# ODE model

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/ODE_Extinct.jl")')
jl.include('/home/callum/ASF/Fitting/ODE_Extinct.jl') #loading files with our model!

#Loading the four models!
model_int = jl.SIR_ODE.model_int #Frequency


In [ ]:
n_sims = 1000
path = "Posteriors/ODE/"


## Intervention Strengths!

Below we are making arrays for the different strengths of intervention ($\alpha$) for the ODE model. Note there is a higher density of $\alpha$ we are testing around the values of the bifurcation (want to see exactly where it occurs)! This point of transition was found by broader initial runs.

In [ ]:
#ODE1
xode1 = np.linspace(0,97.5,21)


In [ ]:
#ODE4
base4 = np.linspace(0,60,13)
mid4 = np.linspace(61,75,140) #area around bifurcation
end4 = np.linspace(72.5,97.5,11)
xode4 = np.concatenate((base4,mid4,end4))

In [ ]:
#ODE2 and ODE3
base2 = np.linspace(0,75,16)
mid2 = np.linspace(76,90,140) #area around bifurcation
end2 = np.linspace(92.5,97.5,3)
xode2 = np.concatenate((base2,mid2,end2))

base3 = np.linspace(0,47,10)
mid3 = np.linspace(47.5,57.5,140) #area around bifurcation
end3 = np.linspace(60,97.5,16)
xode3 = np.concatenate((base3,mid3,end3))

In [ ]:
#Running Interventions
ode1 = simulate_intervention(1,n_sims,path,xode1)
print("done")
ode2 = simulate_intervention(2,n_sims,path,xode2)
print("done")
ode3 = simulate_intervention(3,n_sims,path,xode3)
print("done")
ode4 = simulate_intervention(4,n_sims,path,xode4)

In [ ]:
#saving output!

np.save("Intervention/ODE/int_ode1",ode1)
np.save("Intervention/ODE/int_ode2",ode2)
np.save("Intervention/ODE/int_ode3",ode3)
np.save("Intervention/ODE/int_ode4",ode4)


In [ ]:
#loading input for plotting!

ode1 = np.load("Intervention/ODE/int_ode1.npy")
ode2 = np.load("Intervention/ODE/int_ode2.npy")
ode3 = np.load("Intervention/ODE/int_ode3.npy")
ode4 = np.load("Intervention/ODE/int_ode4.npy")

In [ ]:
#loading credible intervals (calculated in R)
lo1 = np.load("Intervention/ODE/lr1.npy")
lo2 = np.load("Intervention/ODE/lr2.npy")
lo3 = np.load("Intervention/ODE/lr3.npy")
lo4 = np.load("Intervention/ODE/lr4.npy")

uo1 = np.load("Intervention/ODE/ur1.npy")
uo2 = np.load("Intervention/ODE/ur2.npy")
uo3 = np.load("Intervention/ODE/ur3.npy")
uo4 = np.load("Intervention/ODE/ur4.npy")

# TAU Homogeneous 

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HOMO_EXTINCT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HOMO_EXTINCT.jl') #loading files with our model!

#Loading the four models!
model_int = jl.SIR_TAU_S.model_int #Frequency

In [ ]:
#converting alpha to correct format 
def run_tau_int(model_n, n_sims, path, test_array):
    store = np.zeros(len(test_array))
    
    for i, j in enumerate(test_array):
    
        rr = (100 - j)/100
        val = run_simulations_inter(path,model_n,n_sims, rr)
        store[i] = val
    return store

In [ ]:
test_strengths = np.linspace(0,95,100) 

path = "Posteriors/Tau-Homogeneous/"


In [ ]:
tau1 = run_tau_int(1,1000,path,test_strengths)
np.save("Intervention/Homogeneous/int_tau1", tau1)
print(1)
tau2 = run_tau_int(2,1000,path,test_strengths)
np.save("Intervention/Homogeneous/int_tau2", tau2)
print(1)
tau3 = run_tau_int(3,1000,path,test_strengths)
np.save("Intervention/Homogeneous/int_tau3", tau3)
print(1)
tau4 = run_tau_int(4,1000,path,test_strengths)
np.save("Intervention/Homogeneous/int_tau4", tau4)


In [ ]:
#loading credible intervals
cl1 = np.load("Intervention/Homogeneous/lr1.npy")
cl2 = np.load("Intervention/Homogeneous/lr2.npy")
cl3 = np.load("Intervention/Homogeneous/lr3.npy")
cl4 = np.load("Intervention/Homogeneous/lr4.npy")

cu1 =  np.load("Intervention/Homogeneous/ur1.npy")
cu2 =  np.load("Intervention/Homogeneous/ur2.npy")
cu3 =  np.load("Intervention/Homogeneous/ur3.npy")
cu4 =  np.load("Intervention/Homogeneous/ur4.npy")

In [ ]:
#loading data
tau1 = np.load("Intervention/Homogeneous/int_tau1.npy")
tau2 = np.load("Intervention/Homogeneous/int_tau2.npy")
tau3 = np.load("Intervention/Homogeneous/int_tau3.npy")
tau4 = np.load("Intervention/Homogeneous/int_tau4.npy")

In [ ]:
#plotting M1 and M2 intervention successes against each other!

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,2))
fig.suptitle('Intervention Success', y=1.1)


ax1.hlines(25,0,100, color = "k", ls='--')

ax1.plot(xode1,ode1, label = r"$\beta \propto 1$", color = colors[0]) 
ax1.plot(xode2,ode2, label = r"$\beta \propto \rho$", color = colors[1]) 
ax1.plot(xode3,ode3, label = r"$\beta \propto \tanh(\rho)$", color = colors[2]) 
ax1.plot(xode4,ode4, label = r"$\beta \propto \sqrt \rho$", color = colors[3]) 
ax1.set_title("ODE")
ax1.set_xlabel("Reduction in decay time (\%)")
ax1.set_ylabel("ASF endemic\nsimulations (\%)")
ax1.fill_between(xode1, lo1*100, uo1*100, color=colors[0], alpha=0.25)
ax1.fill_between(xode2, lo2*100, uo2*100, color=colors[1], alpha=0.25)

ax1.fill_between(xode3, lo3*100, uo3*100,color=colors[2], alpha=0.25)
ax1.fill_between(xode4, lo4*100, uo4*100,color=colors[3], alpha=0.25)
ax1.set_xlim([0,100])

test_strengths = np.linspace(0,97.5,100)

ax2.hlines(25,0,100, color = "k", ls='--')
ax2.plot(test_strengths,tau1/10, label = r"$C = 1$", color = colors[0]) 
ax2.plot(test_strengths,tau2/10, label = r"$C \propto \rho_N$", color = colors[1]) 
ax2.plot(test_strengths,tau3/10, label = r"$C \propto \tanh (\rho_N)$", color = colors[2]) 
ax2.plot(test_strengths,tau4/10, label = r"$C \propto \sqrt{\rho_N}$", color = colors[3]) 
ax2.set_title("Tau-Leaping Homogeneous")
ax2.set_xlabel("Reduction in decay time (\%)")

ax2.fill_between(test_strengths, cl1*100, cu1*100,
                 color=colors[0], alpha=0.25)

ax2.fill_between(test_strengths, cl4*100, cu4*100,
                 color=colors[3], alpha=0.25)
ax2.fill_between(test_strengths, cl2*100, cu2*100,
                 color=colors[1], alpha=0.25)

ax2.fill_between(test_strengths, cl3*100, cu3*100,
                 color=colors[2], alpha=0.25)

ax2.legend(fontsize=7)
ax2.set_xlim([0,100])
plt.savefig('m1m2inter.pdf', format='pdf', bbox_inches='tight')
ax1.set_ylabel("ASF endemic\nsimulations (\%)")


In [ ]:
#similar but for cycle vs constant 

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,2))
fig.suptitle('Intervention Success', y=1.1)


ax1.hlines(25,0,100, color = "k", ls='--')

ax1.plot(xode1,ode1,  color = colors[0], label = r"Split: $C =1 $") 
ax1.plot(xode1,ode1l,  color = colors[0],ls='--', label = r"Logistic: $C =1 $") 
ax1.plot(xode4,ode4, color = colors[3], label  = r"Split: $C \propto \sqrt{\rho_N}$") 
ax1.plot(lxode4,ode4l, color = colors[3],label = r"Logistic: $C \propto \sqrt{\rho_N}$",ls='--') 



ax1.set_title("ODE")
ax1.set_xlabel("Reduction in decay time (\%)")
ax1.set_ylabel("ASF endemic\nsimulations (\%)")
ax1.fill_between(xode1, lo1*100, uo1*100, color=colors[0], alpha=0.25)
ax1.fill_between(xode1, llo1*100, luo1*100, color=colors[0], alpha=0.25)

ax1.fill_between(xode4, lo4*100, uo4*100,color=colors[3], alpha=0.25)
ax1.fill_between(lxode4, llo4*100, luo4*100,color=colors[3], alpha=0.25)

ax1.set_xlim([0,100])

test_strengths = np.linspace(0,97.5,100)

ax2.hlines(25,0,100, color = "k", ls='--')
ax2.plot(test_strengths,tau1/10, label = r"$C = 1$", color = colors[0]) 
ax2.plot(test_strengths,ltau1/10, label = r"$C = 1$", color = colors[0], ls="--") 
ax2.plot(test_strengths,tau4/10, label = r"$C \propto \sqrt{\rho_N}$", color = colors[3])
ax2.plot(test_strengths,ltau4/10, label = r"$C \propto \sqrt{\rho_N}$", color = colors[3], ls="--") 


ax2.set_title("Tau-Leaping Homogeneous")
ax2.set_xlabel("Reduction in decay time (\%)")

ax2.fill_between(test_strengths, cl1*100, cu1*100,
                 color=colors[0], alpha=0.25)
ax2.fill_between(test_strengths, lcl1*100, lcu1*100,
                 color=colors[0], alpha=0.25)



ax2.fill_between(test_strengths, cl4*100, cu4*100,
                 color=colors[3], alpha=0.25)


ax2.fill_between(test_strengths, lcl4*100, lcu4*100,
                 color=colors[3], alpha=0.25)

ax1.legend(fontsize=7, loc = "center left")
ax2.set_xlim([0,100])
plt.savefig('log_m1m2inter.pdf', format='pdf', bbox_inches='tight')

## Same but for M3 Simualtions

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HETRO_Extinct.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HETRO_Extinct.jl') #loading files with our model!
#again network is deined in extinct file!
#Loading the four models!
model_int = jl.SIR_TAU_I.model_int #Frequency

In [ ]:
def simulate_intervention(path, mo_no, n_sims, network):
    cs = np.linspace(.1,1, 19)[::-1]
    dummy  = np.zeros(len(cs)) 
    for i, j in enumerate(cs):

        val = run_simulations_inter(path, mo_no, n_sims, j, network=network)
        red = val/n_sims
        dummy[i] = red
        
    return dummy

In [ ]:
n_sims = 500

In [ ]:
#Random!
path = "Posteriors/Tau-Hetrogeneous/Random/"
network = 'r'
mo_no = 1
r1_int = simulate_intervention(path, mo_no, n_sims, network)
print("Done")
mo_no = 4
r4_int = simulate_intervention(path, mo_no, n_sims, network)

In [ ]:
np.save("r1_int",r1_int)
np.save("r4_int",r4_int)

In [ ]:
#Scale Free!
path = "Posteriors/Tau-Hetrogeneous/Scale_Free/"
network = 'sf'
mo_no = 1
sf1_int = simulate_intervention(path, mo_no, n_sims, network)
print("Done")
mo_no = 4
sf4_int = simulate_intervention(path, mo_no, n_sims, network)

In [ ]:
np.save("sf1_int",sf1_int)
np.save("sf4_int",sf4_int)

In [ ]:
#loading output

r1_int  = np.load("interventionm3/r1_int5.npy")
r4_int  = np.load("interventionm3/r4_int5.npy")
sf1_int = np.load("interventionm3/sf1_int5.npy")
sf4_int = np.load("interventionm3/sf4_int5.npy")
sw1_int = np.load("interventionm3/sw1_int5.npy")
sw4_int = np.load("interventionm3/sw4_int5.npy")

In [ ]:
ur1 = np.load("interventionm3/r1_iu.npy")
lr1 = np.load("interventionm3/r1_il.npy")

ur4 = np.load("interventionm3/r4_iu.npy")
lr4 = np.load("interventionm3/r4_il.npy")

usf1 = np.load("interventionm3/sf1_iu.npy")
lsf1 = np.load("interventionm3/sf1_il.npy")

usf4 = np.load("interventionm3/sf4_iu.npy")
lsf4 = np.load("interventionm3/sf4_il.npy")

usw1 = np.load("interventionm3/sw1_iu.npy")
lsw1 = np.load("interventionm3/sw1_il.npy")

usw4 = np.load("interventionm3/sw4_iu.npy")
lsw4 = np.load("interventionm3/sw4_il.npy")

In [ ]:
fig = plt.figure(figsize=(4,5))

gs = GridSpec(3, 1, figure=fig, hspace=.5)
plt.suptitle("Tau-Leaping Heterogeneous Intervention Success", y=0.975)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[2, 0])

x = 100*(1- np.linspace(.1,1, 37)[::-1,])

ax1.plot(x, 100*r1_int,color=colors[0], label = r"$C = 1$ ({}\%)".format(r1_int[0]*100))
ax1.plot(x, 100*r4_int,color=colors[3], label = r"$C \propto \sqrt{\rho_N}$"+" ({}\%)".format(r4_int[0]*100))


ax1.fill_between(x, 100*lr1, 100*ur1,color=colors[0], alpha=0.25)
ax1.fill_between(x, 100*lr4, 100*ur4,color=colors[3], alpha=0.25)
ax1.hlines(25,min(x),max(x), color = "k", ls='--')
ax1.set_title("Random Network")
ax1.set_ylabel("ASF endemic\nsimulations (\%)")

ax2.plot(x, 100*sf1_int,color=colors[0], label = r"$C = 1$ ({}\%)".format(sf1_int[0]*100))
ax2.plot(x, 100*sf4_int,color=colors[3], label = r"$C \propto \sqrt{\rho_N}$"+" ({}\%)".format(sf4_int[0]*100))

ax2.fill_between(x, 100*lsf1, 100*usf1,color=colors[0], alpha=0.25)
ax2.fill_between(x, 100*lsf4, 100*usf4,color=colors[3], alpha=0.25)
ax2.hlines(25,min(x),max(x), color = "k", ls='--')
ax2.set_title("Scale-Free Network")
ax2.set_ylabel("ASF endemic\nsimulations (\%)")
ax3.plot(x, 100*sw1_int,color=colors[0], label = r"$C = 1$ ({}\%)".format(np.round(sw1_int[0]*100,1)))
ax3.plot(x, 100*sw4_int,color=colors[3],label = r"$C \propto \sqrt{\rho_N}$"+" ({}\%)".format(sw4_int[0]*100))


ax3.fill_between(x, 100*lsw1, 100*usw1,color=colors[0], alpha=0.25)
ax3.fill_between(x, 100*lsw4, 100*usw4,color=colors[3], alpha=0.25)
ax3.hlines(25,min(x),max(x), color = "k", ls='--')
ax3.set_title("Small-World Network")
ax3.set_xlabel("Reduction in decay time (\%)")
ax3.set_ylabel("ASF endemic\nsimulations (\%)")



ax1.legend(loc="lower left", fontsize=8, framealpha=1)
ax2.legend(loc="lower left",fontsize=8, framealpha=1)
ax3.legend(loc="lower left",fontsize=8, framealpha=1)

ax1.set_frame_on(True) # make sure there is any background

ax1.set_ylim([0,75])
ax1.set_xlim([0,90])
ax2.set_ylim([0,75])
ax2.set_xlim([0,90])
ax3.set_ylim([0,75])
ax3.set_xlim([0,90])
plt.savefig('m3inter.pdf', format='pdf', bbox_inches='tight')
